In [2]:
import pandas as pd
import sqlite3
from create_database_instance import db, update_movie, insert_director, look_for_duplicates, replace_none_image_column, return_duplicates, delete_duplicates
import requests


In [3]:
get_movies = """
SELECT * FROM Movies
"""

In [4]:
url_path = ("https://imdb-api.com/en/API/Title/k_br0y2jj3/")

In [5]:
db.call_db(replace_none_image_column)

[]

In [28]:
df1 = db.call_db(get_movies)
df2 = pd.DataFrame(df1)
df3 = df2[425:426]
df3

,0,1,2,3,4,5,6,7
425,426,426,Hannibal,351692268,2001,tt2243973,0,0


In [11]:
def check_image_int(image):
    try:
        image_col = int(image)
        return image_col
    except: 
        pass


def check_dir_int(director):
    try:
        dir_col = int(director)
        return dir_col
    except:
        pass

def check_for_none(number_check):
    for arg in number_check:
        try:
            arg
            print("test_number_check succeded!")
            return arg
        except:
            print("test_number_check failed")
            pass

Consider removing the none-check. Some movies return none values and break the chain.

In [29]:
max_requests_per_day = 99
counter = 0
num_of_dir_added = 0
num_of_movies_added = 0


for num, movie_id in enumerate(df3[5]):
    image_col_value = df3.iloc[num, 6]
    dir_col_value = df3.iloc[num, 7]
    image = check_image_int(image_col_value)
    dir = check_dir_int(dir_col_value)
    # Check if the value in any column is 0
    if image == 0 or dir == 0:
        resp = requests.get(f"{url_path}{movie_id}")
        print(movie_id)
        counter += 1
        if counter == max_requests_per_day:
            break
        else:
            pass
        resp_json = resp.json()
        try:
            image = resp_json['image']
        except:
            image = 0
        # If none values exist, request limit has been reached. Abort.
        if resp_json['directorList'] is None:
            
            print(f"{movie_id} was the end of the line")
            break
        else:
            # Add directors names to list
            # If no name available, add 0
            directors_name_list = []
            directors_id_list = []
            for i in range(len(resp_json['directorList'])):
                try:
                    directors_name_list.append(resp_json['directorList'][i]['name'])
                except:
                    directors_name_list.append(0)
                try:
                    directors_id_list.append(resp_json['directorList'][i]['id'])
                except:
                    directors_id_list.append(0)
                    # Add each item in the lists to directoras table

            for num, i in enumerate(directors_name_list):
                check_dir_duplicates = db.call_db(look_for_duplicates, i)
                if len(check_dir_duplicates) > 0:
                    # print("duplicates found. Insertion aborted")
                    pass
                else:
                    db.call_db(insert_director, directors_id_list[num], directors_name_list[num])
                    num_of_dir_added += 1
                    # print("Values inserted")

            if len(directors_name_list) > 1:
                director_movie_table = ", ".join(directors_name_list)
                db.call_db(update_movie, image, director_movie_table, movie_id)
                num_of_movies_added += 1
                # print("More directors")
            elif len(directors_name_list) == 1:
                db.call_db(update_movie, image, directors_name_list[0], movie_id)
                num_of_movies_added += 1


print(f"Movies added: {num_of_movies_added}")
print(f"Directors added: {num_of_dir_added}")


tt2243973
Movies added: 0
Directors added: 0


In [30]:
for num, movie_id in enumerate(df3[5]):
    image_col_value = df3.iloc[num, 6]
    dir_col_value = df3.iloc[num, 7]
    image = check_image_int(image_col_value)
    dir = check_dir_int(dir_col_value)
    # Check if the value in any column is 0

    resp = requests.get(f"{url_path}{movie_id}")
    resp_json = resp.json()

In [35]:
resp_json

{'id': 'tt2243973',
 'title': 'Hannibal',
 'originalTitle': '',
 'fullTitle': 'Hannibal (TV Series 2013–2015)',
 'type': 'TVSeries',
 'year': '2013',
 'image': 'https://m.media-amazon.com/images/M/MV5BNWU5YmJhNWEtZGVlOS00NmRhLThmNzUtYjAyMDA2MDg5ZTM4XkEyXkFqcGdeQXVyNDIzMzcwNjc@._V1_Ratio0.7117_AL_.jpg',
 'releaseDate': '2013-04-04',
 'runtimeMins': None,
 'runtimeStr': None,
 'plot': 'Explores the early relationship between renowned psychiatrist Hannibal Lecter and a young FBI criminal profiler who is haunted by his ability to empathize with serial killers.',
 'plotLocal': '',
 'plotLocalIsRtl': False,
 'awards': 'Top rated TV #226 | Nominated for 1 Primetime Emmy, 24 wins & 52 nominations total',
 'directors': '',
 'directorList': [],
 'writers': '',
 'writerList': [],
 'stars': 'Hugh Dancy, Mads Mikkelsen, Caroline Dhavernas',
 'starList': [{'id': 'nm0199215', 'name': 'Hugh Dancy'},
  {'id': 'nm0586568', 'name': 'Mads Mikkelsen'},
  {'id': 'nm0223518', 'name': 'Caroline Dhavernas'}],
